In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
class SAP:
    """SAPの性能から、どれくらいの面積・質量が必要になるかを計算してくれる有能な奴にしたい。
    To do: 温度の影響
    """
    sunlight_strength = 1350 #太陽光強度[W/m^2]
    def __init__(self, Pd, Td=12., Pe=0, Pm=0, Te=12, Tm=0, Xc=0.9, Xd=0.9,
                 vol_degrade_per_decade=0.92, current_degrade_per_decade=0.84,
                 pannel_width=2.2, pannel_length=1.5,
                 cell_efficiency = 0.185, cell_packing_efficiency=0.95,):
        """
        Attribute:
            Pd[W] 日照時消費電力
            Td[hour] 日照時間 default=12
            Pe[W] 食事消費電力 default=0
            Pm[W] ミッション時増加電力 default=0
            Te[hour] 食時間 default=12
            Tm[hour] ミッション時間 default=12
            Xc[] バッテリ-負荷間の電力伝達効率 default=0.9
            Xd[] 太陽電池アレイ-負荷間の電力伝達効率　default=0.9
            
            vol_degrade_per_decade[] 動作電圧劣化 default=0.92
            current_degrade[] 動作電流劣化 default=0.84
            
            pannel_width[m] パネル一枚の幅　default=2.2
            pannel_length[m] パネル一枚の長さ　default=1.5
            
            cell_efficienvy[] セル効率 default=0.185
            cell_packing_efficiency[] セルパッキング効率 default=0.95
            
        """
        self.Pd = Pd
        
        self.vol_degrade_per_decade = vol_degrade_per_decade
        self.current_degrade_per_decade = current_degrade_per_decade
        
        self.cell_efficiency = cell_efficiency
        self.cell_packing_efficiency = cell_packing_efficiency
        
        self.Td = Td
        self.Pe = Pe
        self.Pm = Pm
        self.Te = Te
        self.Tm = Tm
        self.Xc = Xc
        self.Xd = Xd
        self.Psa = (Pe*Te/Xc + Pm*Tm/Xc + Pd*Td/Xd)/Td
        
        self.pannel_squere = pannel_width*pannel_length
        
    def calc_array(self, theta, time_after=100., t_effect=0.):
        """入射角に対してアレイの諸元を決定する。
        Arg:
            theta[deg] 太陽光の入射角
            time_after[yr] n年後 default=100
            t_effect[] 温度影響　default=0
        Return:
            solloar_battery_array_mass[kg] 太陽電池アレイ重量
        """
        print(self.Psa)
        theta = np.deg2rad(theta)#[rad]
        eta = time_after/10.
        degrade_rate = (self.vol_degrade_per_decade*self.current_degrade_per_decade)**eta
        #アレイ発電量
        bol = self.Psa/degrade_rate/np.cos(theta)/(1.-t_effect)
        #全セル面積[m^2]
        all_cell = bol/SAP.sunlight_strength/self.cell_efficiency
        #全セル枚数[枚] 2cm*4cmとする
        cell_number = all_cell/(2*10**(-2)*4*10**(-2))
        #アレイ面積[m^2]
        array = all_cell/self.cell_packing_efficiency
        #翼数
        wing_number = int(np.ceil(array/self.pannel_squere))
        if wing_number & 1:
            wing_number += 1
        else:
            pass
        
        #構造重量(リジッド型パネル2.1kg/m^2)
        pannel_structure_mass = 2.1*(self.pannel_squere)*wing_number
        #展開機構(0.4kg*パネル枚数+2.1kg)
        deployment_structure_mass = 0.4*wing_number + 2.1
        #保持解放機構(9kg・・・要出典)
        holding_release_structure_mass = 9
        #パネル質量
        pannel_mass = pannel_structure_mass+deployment_structure_mass+holding_release_structure_mass
        
        #セル重量 セルの面密度87mg/cm**2, ワイヤハーネス0.2g/個
        cell_mass = cell_number*(87*4*2)*10**(-6) + 0.2*cell_number*10**(-3)
        
        #モータ重量 10kgとする
        motor_mass = 10
        
        #太陽電池アレイ質量
        sollar_battery_array_mass = pannel_mass + cell_mass + motor_mass
        return sollar_battery_array_mass

In [5]:
debug1 = SAP(50)
debug1.calc_array(0)

55.55555555555555


39.03920740934309

In [6]:
11.3/8*10**4

14125.0